# Export to DuckDB

In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

import duckdb

### Load iris data

In [2]:
con = duckdb.connect("../test.db")
cal_house = con.sql("SELECT * FROM cal_house").df().drop(columns=['id'])
con.close()

cal_house.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Value
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.880001,-122.230003,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.860001,-122.220001,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.849998,-122.239998,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.849998,-122.250000,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.849998,-122.250000,3.422


In [16]:
cal_house.iloc[2045:2050, :]

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Value
2045,1.4007,7.0,4.466424,1.174229,1587.0,2.880218,36.730000,-119.730003,2.250
2046,1.4646,9.0,4.514151,1.158019,1116.0,2.632076,36.730000,-119.720001,0.659
2047,6.8162,15.0,7.383621,1.060345,766.0,3.301724,36.720001,-119.720001,1.272
2048,0.6991,26.0,2.669021,1.014127,1660.0,1.675076,36.720001,-119.730003,0.895
2049,2.2437,9.0,4.114213,1.086294,678.0,1.720812,36.730000,-119.730003,0.542


In [3]:
cal_house_data = cal_house.iloc[:, :8]
cal_house_value = cal_house.iloc[:, 8]

### Train the model

In [4]:
x = torch.FloatTensor(cal_house_data.values)
y = torch.FloatTensor(cal_house_value.values).reshape(-1, 1)

In [5]:
model = model = nn.Sequential(
    nn.Linear(8, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

In [6]:
learning_rate = 0.0001
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
def train_network(model, optimizer, loss_fn, X_train, y_train, num_epochs, train_losses):
    for epoch in range(num_epochs):
        #clear out the gradients from the last step loss.backward()
        optimizer.zero_grad()
        
        #forward feed
        output_train = model(X_train)

        #calculate the loss
        loss_train = loss_fn(output_train, y_train)

        #backward propagation: calculate gradients
        loss_train.backward()

        #update the weights
        optimizer.step()

        train_losses[epoch] = loss_train.item()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss_train.item():.4f}")

In [8]:
num_epochs = 1000
train_losses = np.zeros(num_epochs)

In [9]:
train_network(model,optimizer,loss_fn,x,y,num_epochs,train_losses)

Epoch 50/1000, Train Loss: 254.0938
Epoch 100/1000, Train Loss: 136.0820
Epoch 150/1000, Train Loss: 68.7756
Epoch 200/1000, Train Loss: 33.5440
Epoch 250/1000, Train Loss: 16.2363
Epoch 300/1000, Train Loss: 8.3917
Epoch 350/1000, Train Loss: 4.9005
Epoch 400/1000, Train Loss: 3.2734
Epoch 450/1000, Train Loss: 2.4597
Epoch 500/1000, Train Loss: 2.0302
Epoch 550/1000, Train Loss: 1.7866
Epoch 600/1000, Train Loss: 1.6406
Epoch 650/1000, Train Loss: 1.5471
Epoch 700/1000, Train Loss: 1.4852
Epoch 750/1000, Train Loss: 1.4434
Epoch 800/1000, Train Loss: 1.4135
Epoch 850/1000, Train Loss: 1.3918
Epoch 900/1000, Train Loss: 1.3757
Epoch 950/1000, Train Loss: 1.3627
Epoch 1000/1000, Train Loss: 1.3523


### Compile as TorchScript

In [10]:
sm = torch.jit.script(model)

In [11]:
sm.save("cal_house.pt")